<a href="https://colab.research.google.com/github/GenaroHacker/creating_chord_collection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Set Up
%%capture
!git clone https://github.com/GenaroHacker/creating_chord_collection.git

In [8]:
# @title Chord
class GuitarChord:
    all_notes = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    open_string_notes = ["E", "B", "G", "D", "A", "E"]

    def __init__(self, root, chord_type, *, starting_fret=0, finger_ascending):
        self.root = root
        self.chord_type = chord_type
        self.starting_fret = starting_fret
        self.finger_ascending = finger_ascending

    def __str__(self):
        return f"({repr(self.root)}, {repr(self.chord_type)}, finger_ascending={self.finger_ascending}, starting_fret={self.starting_fret})"

    def get_notes(self, include_octaves=False):
        string_notes = []
        notes_with_octaves = {}
        base_octaves = [4, 3, 3, 3, 2, 2]  # Base octaves for standard tuning

        for string_number, (open_note, finger_position) in enumerate(zip(GuitarChord.open_string_notes, self.finger_ascending), start=1):
            if finger_position is None:
                note = None
                octave = None
            else:
                # Calculate the note
                note_index = (GuitarChord.all_notes.index(open_note) + finger_position) % len(GuitarChord.all_notes)
                note = GuitarChord.all_notes[note_index]

                # Calculate the octave
                base_octave = base_octaves[string_number - 1]
                octave = base_octave + (finger_position // 12)

                # Increment the octave if we cross a B to C transition
                if "B" in GuitarChord.all_notes[GuitarChord.all_notes.index(open_note):] and note == "C":
                    octave += 1

            string_notes.append(note)
            notes_with_octaves[string_number] = (note, octave)

        return notes_with_octaves if include_octaves else string_notes

    def is_open(self):
        return 0 in self.finger_ascending

    def get_absolute_notes(self):
        string_notes = self.get_notes()
        filtered_notes = filter(None, string_notes)  # Remove None values
        unique_notes = list(dict.fromkeys(filtered_notes))  # Remove duplicates

        # Create a list starting from the root note
        root_index = GuitarChord.all_notes.index(self.root)
        sorted_notes_order = GuitarChord.all_notes[root_index:] + GuitarChord.all_notes[:root_index]

        # Sort the notes as per their order in sorted_notes_order
        sorted_notes = sorted(unique_notes, key=lambda x: sorted_notes_order.index(x))
        return sorted_notes




# Example usage with explicit keyword arguments
chord = GuitarChord('B', 'm', finger_ascending=[0, 0, 0, 2, 2, 0], starting_fret=0)
#print(chord)
print("String Notes with Octaves:", chord.get_notes(include_octaves=True))
print("Is Open Chord:", chord.is_open())
print("Absolute Notes:", chord.get_absolute_notes())



String Notes with Octaves: {1: ('E', 4), 2: ('B', 3), 3: ('G', 3), 4: ('E', 3), 5: ('B', 2), 6: ('E', 2)}
Is Open Chord: True
Absolute Notes: ['B', 'E', 'G']


In [3]:
# @title Collection
import sqlite3
from collections import defaultdict

class ChordCollection:
    def __init__(self):
        self.chords = defaultdict(list)

    def load(self, db_path):
        self.chords.clear()
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()

        cursor.execute('SELECT ROOT, TYPE, STARTING_FRET, STRING_1, STRING_2, STRING_3, STRING_4, STRING_5, STRING_6 FROM TABLE_CHORDS')
        rows = cursor.fetchall()

        for row in rows:
            root, chord_type, starting_fret, *fingers = row
            chord = GuitarChord(root, chord_type, finger_ascending=fingers, starting_fret=starting_fret)
            self.chords[root].append(chord)

        connection.close()

# Example usage
chord_collection = ChordCollection()
chord_collection.load('/content/creating_chord_collection/chord_collection.db')
chord = chord_collection.chords["E"][6]
print(chord)
print("String Notes with Octaves:", chord.get_notes(include_octaves=True))
print("Is Open Chord:", chord.is_open())
print("Absolute Notes:", chord.get_absolute_notes())



('E', 'm', finger_ascending=[0, 0, 0, 2, 2, 0], starting_fret=None)
String Notes with Octaves: {1: ('E', 4), 2: ('B', 3), 3: ('G', 3), 4: ('E', 3), 5: ('B', 2), 6: ('E', 2)}
Is Open Chord: True
Absolute Notes: ['E', 'G', 'B']
